In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as scist

import differential_expression_results_analysis as diffexpr
import genes_correlation as gc
import deconvolution_utils as deconv
import predictive_model as predm
import plot_pictures as pict

#### I. RNA-Seq data analysis of the DCs (co-cultured with glioma GL261 cells) treated with either PS-PDT and MTX 

*a) Prepare data, Normalization and Differential Expression Analysis* 

Input data files:
- DC-DCPS-DCMTX-counts.csv
- DC-DCPS-DCMTX-sample-attributes.csv
- GCF_000001635.27_GRCm39_feature_table.txt

R script (prepare_normalization_DESeq2.R) generates the following files: 
- DC-DCPS-DCMTX-counts-protein-coding-genes.csv
- DC-DCPS-DCMTX-log2TPMnorm-data.csv
- DESeq2res-DCPS-vs-DC.csv
- DESeq2res-DCMTX-vs-DC.csv

*instead "path" -> path to data folder with files*

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" prepare_normalization_DESeq2.R "path" "data\DC-DCPS-DCMTX-counts.csv" "data\DC-DCPS-DCMTX-sample-attributes.csv" "data\GCF_000001635.27_GRCm39_feature_table.txt" 1 "" ""

In [ ]:
DESeq2_results_psvsdc = pd.read_csv('data/DESeq2res-DCPS-vs-DC.csv', sep=';')
DESeq2_results_mtxvsdc = pd.read_csv('data/DESeq2res-DCMTX-vs-DC.csv', sep=';')

deg_ps = diffexpr.deg_identification(DESeq2_results_psvsdc, pval_thr=0.05, baseMean_thr=100)
deg_ps_fc2 = diffexpr.deg_identification(DESeq2_results_psvsdc, pval_thr=0.05, baseMean_thr=100, log2FC_thr=1)
deg_ps_fc2_up, deg_ps_fc2_down = diffexpr.up_down_deg(deg_ps_fc2)
print(f'DEG (|FC|>2) for DCPS vs DC:\n\tall = {len(deg_ps_fc2)}, up = {len(deg_ps_fc2_up)}, down = {len(deg_ps_fc2_down)}')

deg_mtx = diffexpr.deg_identification(DESeq2_results_mtxvsdc, pval_thr=0.05, baseMean_thr=100)
deg_mtx_fc2 = diffexpr.deg_identification(DESeq2_results_mtxvsdc, pval_thr=0.05, baseMean_thr=100, log2FC_thr=1)
deg_mtx_fc2_up, deg_mtx_fc2_down = diffexpr.up_down_deg(deg_mtx_fc2)
print(f'DEG (|FC|>2) for DCMTX vs DC:\n\tall = {len(deg_mtx_fc2)}, up = {len(deg_mtx_fc2_up)}, down = {len(deg_mtx_fc2_down)}')

common_genes = diffexpr.common_deg('DCPS', deg_ps, 'DCMTX', deg_mtx)
common_genes_fc2 = diffexpr.common_deg('DCPS', deg_ps_fc2, 'DCMTX', deg_mtx_fc2)
common_genes_fc2_up = diffexpr.common_deg('DCPS', deg_ps_fc2_up, 'DCMTX', deg_mtx_fc2_up)
common_genes_fc2_down = diffexpr.common_deg('DCPS', deg_ps_fc2_down, 'DCMTX', deg_mtx_fc2_down)
print(f'common DEG (|FC|>2) for DCPS and DCMTX:\n\tall = {len(common_genes_fc2)}, \n\tsimultaneously up = {len(common_genes_fc2_up)}, \n\tsimultaneously down = {len(common_genes_fc2_down)}, \n\tbehave differently = {len(common_genes_fc2)-len(common_genes_fc2_up)-len(common_genes_fc2_down)}')


pict.plot_venn_diagram_deg(len(deg_mtx_fc2), len(deg_ps_fc2), len(common_genes_fc2), ['DC+MTX', 'DC+PS-PDT'], 'pictures/venn_diagram_for_deg_ps_mtx')

ps_fc = diffexpr.get_fold_changes(common_genes['log2FoldChange_DCPS'].values)
mtx_fc = diffexpr.get_fold_changes(common_genes['log2FoldChange_DCMTX'].values)

pict.plot_hist_deg(ps_fc, mtx_fc, 'pictures/histogramm_for_deg_ps_mtx')

*b) Correlation analysis of markers of DC maturation*

In [ ]:
marker_genes = pd.read_csv('data/61_marker_genes.csv')['gene_name'].values

countsDataNorm = pd.read_csv('data/DC-DCPS-DCMTX-log2TPMnorm-data.csv', sep=';')
countsDataNorm = countsDataNorm[countsDataNorm['gene_name'].isin(marker_genes)]
countsDataNorm.reset_index(drop=True, inplace=True)

countsDataNorm_ps = countsDataNorm[['gene_name', 'DC1_PS', 'DC2_PS', 'DC3_PS', 'DC4_PS']]
corr_matrix_ps = gc.correlation(countsDataNorm_ps)
corr_matrix_ps.to_csv('data/corr_PS_for_61_marker_genes.csv', sep=';')

countsDataNorm_mtx = countsDataNorm[['gene_name', 'DC1_MTX', 'DC2_MTX', 'DC3_MTX', 'DC4_MTX']]
corr_matrix_mtx = gc.correlation(countsDataNorm_mtx)
corr_matrix_mtx.to_csv('data/corr_MTX_for_61_marker_genes.csv', sep=';')

*Plot heatmaps*

*instead "path" -> path to data folder with files*

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" plot_heatmaps_for_DCmarker_genes.R "path" "data\corr_PS_for_61_marker_genes.csv" "data\corr_MTX_for_61_marker_genes.csv"

*c) Th17 cells*

In [ ]:
genes = ['Tgfb3', 'Il6', 'Il23a']
countsDataNorm = pd.read_csv('data/DC-DCPS-DCMTX-log2TPMnorm-data.csv', sep=';')
countsDataNorm = countsDataNorm[countsDataNorm['gene_name'].isin(genes)]
countsDataNorm = countsDataNorm.T
countsDataNorm.columns = countsDataNorm.iloc[0].values
countsDataNorm.drop(['gene_name'], axis=0, inplace=True)
countsDataNorm['Group'] = countsDataNorm.index
countsDataNorm.reset_index(drop=True, inplace=True)

DESeq2_results_psvsdc = pd.read_csv('data/DESeq2res-DCPS-vs-DC.csv', sep=';')
DESeq2_results_mtxvsdc = pd.read_csv('data/DESeq2res-DCMTX-vs-DC.csv', sep=';')

pict.plot_boxplots_th17cells(genes, countsDataNorm, DESeq2_results_psvsdc, DESeq2_results_mtxvsdc, 'pictures/Th17_genes')

#### II. RNA-Seq data analysis of the TCGA-LGG project

*a) Prepare data, Normalization and Differential Expression Analysis* 

Input Data files:
- TCGA-LGG-counts.csv
- TCGA-LGG-sample-attributes.csv
- GEO-42-controls-counts.csv
- GEO-42-controls-sample-attributes
- gencode.v36.annotation.gtf

R script (prepare_normalization_DESeq2.R) generates the following files: 
- TCGA-LGG-counts-protein-coding-genes.csv
- TCGA-LGG-log2TPMnorm-data.csv
- DESeq2res-Dead-vs-Alive.csv
- DESeq2res-Alive-vs-Control.csv
- DESeq2res-Dead-vs-Control.csv

*instead "path" -> path to data folder with files*

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" batch_correction.R "path" "GEO" "data\GEO-42-controls-counts.csv"

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" prepare_normalization_DESeq2.R "path" "data\TCGA-LGG-counts.csv" "data\TCGA-LGG-sample-attributes.csv" "data\gencode.v36.annotation.gtf" 2 "data\GEO-42-controls-ComBatSeq-counts.csv" "data\GEO-42-controls-sample-attributes.csv"

*b) Calculation of correlations for Th17 cells*

In [ ]:
genes_for_tcells = {
    'Th17': ['IL21', 'IL26', 'IL17A', 'IL17F', 'RORA', 'RORC', 'CCR6', 'IL23A', 'KLRB1', 'IL6', 'TNF', 'IL1B', 'CD5', 'CD6',
             'CCL20', 'CXCR3', 'IL1R1', 'IL1R2', 'TNFRSF1A', 'TNFRSF1B', 'IL17RA', 'IL17RC', 'CCR2', 'IL8', 'LCN2', 'DEFB4A',
             'CXCL1', 'CXCL2', 'CCL2', 'CCR1', 'CXCL5', 'CXCL6'] }

data = pd.read_csv('data/TCGA-LGG-log2TPMnorm-data.csv', sep=';')

corr_matrix = gc.correlation(data[data['gene_name'].isin(genes_for_tcells['Th17'])].reset_index(drop=True))
corr_matrix.to_csv(f'data/TCGA-LGG-Th17-corr-matrix.csv', sep=';')

*Plot heatmaps*

*instead "path" -> path to data folder with files*

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" plot_heatmaps_for_tcells.R "path"

*c) Analysis of Th17 cells metagene*

In [ ]:
cell_types = ['Th17']

genes_for_metagene = {
    'Th17': ['LCN2', 'DEFB4A', 'IL17A']
}

data = pd.read_csv('data/TCGA-LGG-log2TPMnorm-data.csv', sep=';')
attributes = pd.read_csv('data/TCGA-LGG-sample-attributes.csv', sep=';')


for ct in cell_types:
    data_ct = data[data['gene_name'].isin(genes_for_metagene[ct])].reset_index(drop=True)
    metagene_expr = pd.DataFrame({'metagene_expr': data_ct.mean(axis=0)})
    metagene_expr['sample'] = metagene_expr.index
    metagene_expr = metagene_expr.merge(attributes, left_on='sample', right_on='sample')
    
    metagene_expr_thr = np.percentile(metagene_expr['metagene_expr'].values, 75)
    
    metagene_expr_low = metagene_expr[metagene_expr['metagene_expr'] <= metagene_expr_thr]
    metagene_expr_high = metagene_expr[metagene_expr['metagene_expr'] > metagene_expr_thr]
    
    pict.overall_surv_for_metagenes(metagene_expr_low, metagene_expr_high, ct, f'pictures/{ct}-metagene-OS')

In [ ]:
clinical = pd.read_csv('data/clinical.tsv', sep='\t')
no_pharma = clinical[(clinical['treatment_or_therapy'] == 'no') & (clinical['treatment_type'] == 'Pharmaceutical Therapy, NOS')]
yes_pharma = clinical[(clinical['treatment_or_therapy'] == 'yes') & (clinical['treatment_type'] == 'Pharmaceutical Therapy, NOS')]
no_radiation = clinical[(clinical['treatment_or_therapy'] == 'no') & (clinical['treatment_type'] == 'Radiation Therapy, NOS')]
yes_radiation = clinical[(clinical['treatment_or_therapy'] == 'yes') & (clinical['treatment_type'] == 'Radiation Therapy, NOS')]

no_treatment_or_therapy = list(set(no_pharma['case_submitter_id'].values) & set(no_radiation['case_submitter_id'].values))
yes_pharma_no_radiation = list(set(yes_pharma['case_submitter_id'].values) & set(no_radiation['case_submitter_id'].values))
no_pharma_yes_radiation = list(set(no_pharma['case_submitter_id'].values) & set(yes_radiation['case_submitter_id'].values))
yes_pharma_yes_radiation = list(set(yes_pharma['case_submitter_id'].values) & set(yes_radiation['case_submitter_id'].values))
yes_treatment_or_therapy = list(set(no_pharma_yes_radiation) | set(yes_pharma_no_radiation) | set(yes_pharma_yes_radiation))

metagene_expr_no = metagene_expr[metagene_expr['sample'].isin(no_treatment_or_therapy)]['metagene_expr']
metagene_expr_yes = metagene_expr[metagene_expr['sample'].isin(yes_treatment_or_therapy)]['metagene_expr']
print(metagene_expr_no.median(), ' vs ', metagene_expr_yes.median())
print(scist.mannwhitneyu(metagene_expr_no, metagene_expr_yes, alternative='less'))
print(scist.mannwhitneyu(metagene_expr_yes, metagene_expr_no, alternative='greater'))


*d) Identification of the gene signature associated with overall survival of glioma patients*

In [ ]:
DSeq2_results_DvsA = pd.read_csv('data/DESeq2res-Dead-vs-Alive.csv', sep=';')
DSeq2_results_AvsC = pd.read_csv('data/DESeq2res-Alive-vs-Control.csv', sep=';')
DSeq2_results_DvsC = pd.read_csv('data/DESeq2res-Dead-vs-Control.csv', sep=';')
DSeq2_results_psvsdc = pd.read_csv('data/DESeq2res-DCPS-vs-DC.csv', sep=';')
DSeq2_results_mtxvsdc = pd.read_csv('data/DESeq2res-DCMTX-vs-DC.csv', sep=';')

unidirectional_genes = predm.identification_genes_for_predictive_model(DSeq2_results_DvsA, DSeq2_results_AvsC, DSeq2_results_DvsC, 
                                                                       DSeq2_results_psvsdc, DSeq2_results_mtxvsdc)
print(f'unidirectional_genes = {unidirectional_genes}')

data = pd.read_csv('data/TCGA-LGG-log2TPMnorm-data.csv', sep=';')
attributes = pd.read_csv('data/TCGA-LGG-sample-attributes.csv', sep=';')
data = predm.take_genes_and_transposition(data, unidirectional_genes)
data = data.merge(attributes, left_on='sample', right_on='sample')
sign_genes = predm.univatiate(unidirectional_genes, data, attributes)
print(f'sign_genes = {sign_genes}')
concordance_index, coefs = predm.multivariate(sign_genes, data, attributes)
print(f'concordance_index = {round(concordance_index, 2)}')
print(coefs)

pict.plot_log2fc_genes_from_predictive_model(sign_genes, DSeq2_results_AvsC, DSeq2_results_DvsC, 
                                             DSeq2_results_psvsdc, DSeq2_results_mtxvsdc, 
                                             'pictures/log2fc_genes_predictive_model')

data['risk_score'] = predm.risk_score(data, coefs)
data.to_csv('data/TCGA-LGG-risk-score-for-4-gene-model.csv', sep=';', index=False)
rs_median = np.median(data['risk_score'].values)
print('rs_median =', rs_median)
data_low = data[data['risk_score'] <= rs_median]
print('data_low.shape:', data_low.shape)
data_high = data[data['risk_score'] > rs_median]
print('data_high.shape:', data_high.shape)

logrank_pval = predm.logrank_test_low_high(data_low, data_high)
print('logrank_pval:', logrank_pval)

pict.plot_risk_score_expr_level_for_genes_in_predictive_model(sign_genes, data_low, data_high, rs_median,
                                                             'pictures/TCGA_LGG_genes_predictive_model_risk_score_zscore')

survivalROC Analysis

*instead "path" -> path to data folder with files*

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" survivalROC_analysis.R "path" "TCGA-LGG" "data/TCGA-LGG-risk-score-for-4-gene-model.csv"

In [ ]:
roc_1y = pd.read_csv('data/TCGA-LGG-ROC-1-year.csv', sep=';')
roc_3y = pd.read_csv('data/TCGA-LGG-ROC-3-year.csv', sep=';')
roc_5y = pd.read_csv('data/TCGA-LGG-ROC-5-year.csv', sep=';')
aucs = pd.read_csv('data/TCGA-LGG-AUC-1-3-5-years.csv', sep=';')

pict.plot_surv_roc_curves_for_predictive_model(data_low, data_high, roc_1y, roc_3y, roc_5y, aucs, 'pictures/TCGA_LGG_surv_roc_curve_for_pred_model')

*e) Cellular Deconvolution*

*instead "path" -> path to data folder with files*

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" cellular_deconvolution.R "TCGA-LGG" "path" "data\TCGA-LGG-log2TPMnorm-data.csv"

In [ ]:
deconv_res = pd.read_csv('data/TCGA-LGG-epic-deconvolution.csv', sep=';')
deconv_res.rename(columns={'Bcells': 'B cells', 'CD4_Tcells': 'T cells CD4+', 'CD8_Tcells': 'T cells CD8+',
                           'Endothelial': 'Endothelial cells', 'NKcells': 'NK cells', 'otherCells': 'Uncharacterized cells'}, inplace=True)

data_ct = data.merge(deconv_res, left_on='sample', right_on='sample')

for cells in deconv_res.columns[1:]:
    pict.correlation_for_cells(data_ct, cells, f'pictures/correlation-for-{cells}')

*f) Predictive Model Validation on CGGA-LGG dataset*

*instead "path" -> path to data folder with files*

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" prepare_additional_datasets.R "path" "CGGA-LGG" "data/CGGA-LGG-batches-counts.csv" "data/gencodev36-protein-coding-gene-lengths.csv"

In [ ]:
data = pd.read_csv('data/CGGA-LGG-log2TPMnorm-data.csv', sep=';')
attributes = pd.read_csv('data/CGGA-LGG-sample-attributes.csv', sep=';')

data = predm.take_genes_and_transposition(data, sign_genes)
data = data.merge(attributes, left_on='sample', right_on='sample')
data['risk_score'] = predm.risk_score(data, coefs)
data.to_csv('data/CGGA-LGG-risk-score-for-4-gene-model.csv', sep=';', index=False)

data_low = data[data['risk_score'] <= rs_median]
print('data_low.shape:', data_low.shape)
data_high = data[data['risk_score'] > rs_median]
print('data_high.shape:', data_high.shape)

logrank_pval = predm.logrank_test_low_high(data_low, data_high)
print('logrank_pval:', logrank_pval)

pict.plot_risk_score_expr_level_for_genes_in_predictive_model(sign_genes, data_low, data_high, rs_median,
                                                             'pictures/CGGA_LGG_genes_predictive_model_risk_score_zscore')

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" survivalROC_analysis.R "path" "CGGA-LGG" "data/CGGA-LGG-risk-score-for-4-gene-model.csv"

In [ ]:
roc_1y = pd.read_csv('data/CGGA-LGG-ROC-1-year.csv', sep=';')
roc_3y = pd.read_csv('data/CGGA-LGG-ROC-3-year.csv', sep=';')
roc_5y = pd.read_csv('data/CGGA-LGG-ROC-5-year.csv', sep=';')
aucs = pd.read_csv('data/CGGA-LGG-AUC-1-3-5-years.csv', sep=';')

pict.plot_surv_roc_curves_for_predictive_model(data_low, data_high, roc_1y, roc_3y, roc_5y, aucs, 'pictures/CGGA_LGG_surv_roc_curve_for_pred_model')

*g) Model validation on glioblastoma data (TCGA-GBM, CGGA-GBM)*

TCGA-GBM

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" prepare_additional_datasets.R "path" "TCGA-GBM" "data/TCGA-GBM-counts.csv" "data/gencodev36-protein-coding-gene-lengths.csv"

In [ ]:
data = pd.read_csv('data/TCGA-GBM-log2TPMnorm-data.csv', sep=';')
attributes = pd.read_csv('data/TCGA-GBM-sample-attributes.csv', sep=';')

data = predm.take_genes_and_transposition(data, sign_genes)
data = data.merge(attributes, left_on='sample', right_on='sample')
data['risk_score'] = predm.risk_score(data, coefs)
data.to_csv('data/TCGA-GBM-risk-score-for-4-gene-model.csv', sep=';', index=False)

data_low = data[data['risk_score'] <= rs_median]
print('data_low.shape:', data_low.shape)
data_high = data[data['risk_score'] > rs_median]
print('data_high.shape:', data_high.shape)

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" survivalROC_analysis.R "path" "TCGA-GBM" "data/TCGA-GBM-risk-score-for-4-gene-model.csv"

In [ ]:
roc_1y = pd.read_csv('data/TCGA-GBM-ROC-1-year.csv', sep=';')
roc_3y = pd.read_csv('data/TCGA-GBM-ROC-3-year.csv', sep=';')
roc_5y = pd.read_csv('data/TCGA-GBM-ROC-5-year.csv', sep=';')
aucs = pd.read_csv('data/TCGA-GBM-AUC-1-3-5-years.csv', sep=';')

pict.plot_surv_roc_curves_for_predictive_model(data_low, data_high, roc_1y, roc_3y, roc_5y, aucs, 'pictures/TCGA-GBM_surv_roc_curve_for_pred_model')

CGGA-GBM

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" prepare_additional_datasets.R "path" "CGGA-GBM" "data/CGGA-GBM-batches-counts.csv" "data/gencodev36-protein-coding-gene-lengths.csv"

In [ ]:
data = pd.read_csv('data/CGGA-GBM-log2TPMnorm-data.csv', sep=';')
attributes = pd.read_csv('data/CGGA-GBM-sample-attributes.csv', sep=';')

data = predm.take_genes_and_transposition(data, sign_genes)
data = data.merge(attributes, left_on='sample', right_on='sample')
data['risk_score'] = predm.risk_score(data, coefs)
data.to_csv('data/CGGA-GBM-risk-score-for-4-gene-model.csv', sep=';', index=False)

data_low = data[data['risk_score'] <= rs_median]
print('data_low.shape:', data_low.shape)
data_high = data[data['risk_score'] > rs_median]
print('data_high.shape:', data_high.shape)

logrank_pval = predm.logrank_test_low_high(data_low, data_high)
print('logrank_pval:', logrank_pval)

pict.plot_risk_score_expr_level_for_genes_in_predictive_model(sign_genes, data_low, data_high, rs_median,
                                                             'pictures/CGGA_GBM_genes_predictive_model_risk_score_zscore')

In [ ]:
%%cmd
"C:\Program Files\R\R-4.1.2\bin\Rscript.exe" survivalROC_analysis.R "path" "CGGA-GBM" "data/CGGA-GBM-risk-score-for-4-gene-model.csv"

In [ ]:
roc_1y = pd.read_csv('data/CGGA-GBM-ROC-1-year.csv', sep=';')
roc_3y = pd.read_csv('data/CGGA-GBM-ROC-3-year.csv', sep=';')
roc_5y = pd.read_csv('data/CGGA-GBM-ROC-5-year.csv', sep=';')
aucs = pd.read_csv('data/CGGA-GBM-AUC-1-3-5-years.csv', sep=';')

pict.plot_surv_roc_curves_for_predictive_model(data_low, data_high, roc_1y, roc_3y, roc_5y, aucs, 'pictures/CGGA_GBM_surv_roc_curve_for_pred_model')